

---

<center><h1>Assignment 4</h1></center>

---

# 1. <font color='#556b2f'> **Support Vector Machines with Synthetic Data**</font>, 50 points. 

For this problem, we will generate synthetic data for a nonlinear binary classification problem and partition it into training, validation and test sets. Our goal is to understand the behavior of SVMs with Radial-Basis Function (RBF) kernels with different values of $C$ and $\gamma$.

In [1]:
# DO NOT EDIT THIS FUNCTION; IF YOU WANT TO PLAY AROUND WITH DATA GENERATION, 
# MAKE A COPY OF THIS FUNCTION AND THEN EDIT
#
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

def generate_data(n_samples, tst_frac=0.2, val_frac=0.2):
  # Generate a non-linear data set
  X, y = make_moons(n_samples=n_samples, noise=0.25, random_state=42)
   
  # Take a small subset of the data and make it VERY noisy; that is, generate outliers
    m = 30
    np.random.seed(30)  # Deliberately use a different seed
    ind = np.random.permutation(n_samples)[:m]
    X[ind, :] += np.random.multivariate_normal([0, 0], np.eye(2), (m, ))
    y[ind] = 1 - y[ind]

    # Plot this data
    cmap = ListedColormap(['#b30065', '#178000'])  
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, edgecolors='k')       

    # First, we use train_test_split to partition (X, y) into training and test sets
    X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=tst_frac, 
                                                random_state=42)

    # Next, we use train_test_split to further partition (X_trn, y_trn) into training and validation sets
    X_trn, X_val, y_trn, y_val = train_test_split(X_trn, y_trn, test_size=val_frac, 
                                                random_state=42)

    return (X_trn, y_trn), (X_val, y_val), (X_tst, y_tst)

IndentationError: unexpected indent (2651250381.py, line 19)

In [ ]:
#
#  DO NOT EDIT THIS FUNCTION; IF YOU WANT TO PLAY AROUND WITH VISUALIZATION, 
#  MAKE A COPY OF THIS FUNCTION AND THEN EDIT 
#

def visualize(models, param, X, y):
  # Initialize plotting
    if len(models) % 3 == 0:
    nrows = len(models) // 3
    else:
    nrows = len(models) // 3 + 1

    fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(15, 5.0 * nrows))
    cmap = ListedColormap(['#b30065', '#178000'])

    # Create a mesh
    xMin, xMax = X[:, 0].min() - 1, X[:, 0].max() + 1
    yMin, yMax = X[:, 1].min() - 1, X[:, 1].max() + 1
    xMesh, yMesh = np.meshgrid(np.arange(xMin, xMax, 0.01), 
                             np.arange(yMin, yMax, 0.01))

    for i, (p, clf) in enumerate(models.items()):
    # if i > 0:
    #   break
    r, c = np.divmod(i, 3)
    ax = axes[r, c]

    # Plot contours
    zMesh = clf.decision_function(np.c_[xMesh.ravel(), yMesh.ravel()])
    zMesh = zMesh.reshape(xMesh.shape)
    ax.contourf(xMesh, yMesh, zMesh, cmap=plt.cm.PiYG, alpha=0.6)

    if (param == 'C' and p > 0.0) or (param == 'gamma'):
        ax.contour(xMesh, yMesh, zMesh, colors='k', levels=[-1, 0, 1], 
                 alpha=0.5, linestyles=['--', '-', '--'])

    # Plot data
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, edgecolors='k')       
    ax.set_title('{0} = {1}'.format(param, p))

In [ ]:
# Generate the data
n_samples = 300    # Total size of data set 
(X_trn, y_trn), (X_val, y_val), (X_tst, y_tst) = generate_data(n_samples)

---
### **a**. (25 points)  The effect of the regularization parameter, $C$
Complete the Python code snippet below that takes the generated synthetic 2-d data as input and learns non-linear SVMs. Use scikit-learn's [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) function to learn SVM models with **radial-basis kernels** for fixed $\gamma$ and various choices of $C \in \{10^{-3}, 10^{-2}\, \cdots, 1, \, \cdots\, 10^5\}$. The value of $\gamma$ is fixed to $\gamma = \frac{1}{d \cdot \sigma_X}$, where $d$ is the data dimension and $\sigma_X$ is the standard deviation of the data set $X$. SVC can automatically use these setting for $\gamma$ if you pass the argument gamma = 'scale' (see documentation for more details).

**Plot**: For each classifier, compute **both** the **training error** and the **validation error**. Plot them together, making sure to label the axes and each curve clearly.

**Discussion**: How do the training error and the validation error change with $C$? Based on the visualization of the models and their resulting classifiers, how does changing $C$ change the models? Explain in terms of minimizing the SVM's objective function $\frac{1}{2} \mathbf{w}^\prime \mathbf{w} \, + \, C \, \Sigma_{i=1}^n \, \ell(\mathbf{w} \mid \mathbf{x}_i, y_i)$, where $\ell$ is the hinge loss for each training example $(\mathbf{x}_i, y_i)$.

**Final Model Selection**: Use the validation set to select the best the classifier corresponding to the best value, $C_{best}$. Report the accuracy on the **test set** for this selected best SVM model. _Note: You should report a single number, your final test set accuracy on the model corresponding to $C_{best}$_.

In [ ]:
# Learn support vector classifiers with a radial-basis function kernel with 
# fixed gamma = 1 / (n_features * X.std()) and different values of C
C_range = np.arange(-3.0, 6.0, 1.0)
C_values = np.power(10.0, C_range)

models = dict()
trnErr = dict()
valErr = dict()
target_class = None

minimum = 1
for C in C_values:

    model = SVC(C=C , kernel = "rbf", gamma = "scale",random_state=0)
    model.fit(X_trn, y_trn)
    models[C]=model
    y_trainpred = model.predict(X_trn)
    y_valpred = model.predict(X_val)
    trnErr[C]=1-accuracy_score(y_trn,y_trainpred)
    valErr[C]=1-accuracy_score(y_val,y_valpred)
    if(valErr[C] < minimum):
        minimum=valErr[C]
        target_class=C
   
  
visualize(models, 'C', X_trn, y_trn)

plt.figure()
plt.plot(np.log10(list(valErr.keys())), list(valErr.values()), marker='X', linewidth=3, markersize=12)
plt.plot(np.log10(list(trnErr.keys())), list(trnErr.values()), marker='D', linewidth=3, markersize=12)
plt.xlabel('10^C', fontsize=18)
plt.ylabel('Validation/Train error', fontsize=18)
plt.xticks(np.log10(list(valErr.keys())), fontsize=12)
plt.legend(['Validation Error', 'Training Error'], fontsize=18)

predict = models[target_class].predict(X_tst)
score = metrics.accuracy_score(y_tst, predict)
print("Accuracy: {0:.2f}%, for C = {1:1.0f}".format(score * 100,target_class))

### Discussion 

With increasing values of 'C', we can observe that the training error decreases but after C=100, the validation error increases and decreases intermittently. Larger values of 'C' yields a smaller margin whereas a smaller value of 'C' yields a wider margin. The increase in validation error is due to overfitting. Large values of C makes missclassification stricter hence, the smaller margin.

In [ ]:
model = SVC(C = 100,gamma='scale',kernel='rbf')
model.fit(X_trn, y_trn)
y_test_pred=model.predict(X_tst)
test_error=1-accuracy_score(y_tst,y_test_pred)
print('Test Error  : '+ str(test_error))
print(accuracy_score(y_tst,y_test_pred)*100)

---
### **b**. (25 points)  The effect of the RBF kernel parameter, $\gamma$
Complete the Python code snippet below that takes the generated synthetic 2-d data as input and learns various non-linear SVMs. Use scikit-learn's [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) function to learn SVM models with **radial-basis kernels** for fixed $C$ and various choices of $\gamma \in \{10^{-2}, 10^{-1}\, 1, 10, \, 10^2 \, 10^3\}$. The value of $C$ is fixed to $C = 10$.

**Plot**: For each classifier, compute **both** the **training error** and the **validation error**. Plot them together, making sure to label the axes and each curve clearly.

**Discussion**: How do the training error and the validation error change with $\gamma$? Based on the visualization of the models and their resulting classifiers, how does changing $\gamma$ change the models? Explain in terms of the functional form of the RBF kernel, $\kappa(\mathbf{x}, \,\mathbf{z}) \, = \, \exp(-\gamma \cdot \|\mathbf{x} - \mathbf{z} \|^2)$

**Final Model Selection**: Use the validation set to select the best the classifier corresponding to the best value, $\gamma_{best}$. Report the accuracy on the **test set** for this selected best SVM model. _Note: You should report a single number, your final test set accuracy on the model corresponding to $\gamma_{best}$_.

In [ ]:
# Learn support vector classifiers with a radial-basis function kernel with 
# fixed C = 10.0 and different values of gamma
gamma_range = np.arange(-2.0, 4.0, 1.0)
gamma_values = np.power(10.0, gamma_range)

models = dict()
trnErr = dict()
valErr = dict()
target_g = None
minimum=1

for G in gamma_values:
    model = SVC(C=10.0, kernel = "rbf", gamma = G)
    models[G] = model.fit(X_trn, y_trn)
    trnErr[G] = 1-models[G].score(X_trn, y_trn)
    valErr[G] = 1-models[G].score(X_val, y_val)
    if valErr[G] < minimum:
        minimum = valErr[G]
        target_g = G
  
visualize(models, 'gamma', X_trn, y_trn)

plt.figure()
plt.plot(np.log10(list(valErr.keys())), list(valErr.values()), marker='o', linewidth=4, markersize=12)
plt.plot(np.log10(list(trnErr.keys())), list(trnErr.values()), marker='s', linewidth=4, markersize=12)
plt.xlabel('Gamma= 10^n', fontsize=18)
plt.ylabel('Validation/Train error', fontsize=18)
plt.xticks(np.log10(list(valErr.keys())), fontsize=12)
plt.legend(['Validation Error', 'Training Error'], fontsize=18)

predict = models[target_g].predict(X_tst)
score = metrics.accuracy_score(y_tst, predict)
print("Accuracy: {0:.2f}%, for 𝛾 = {1:1.0f}".format(score * 100,target_g))

### Discussion

The training error decreases for increasing values of gamma however, the validation error decreases for smaller values of gamma and correspondingly increases for higher values of gamma indicating and overfitting of the model. 
Low gamma values imply broader decision regions and higher gamma values imply closer influence.

In [ ]:
model = SVC(C = 10,gamma=1.0,kernel='rbf')
model.fit(X_trn, y_trn)
y_test_pred = model.predict(X_tst)
testErr = accuracy_score(y_tst,y_test_pred)*100
print('Test Error was evaluated to be : '+ str(testErr)+' %')

---
# 2. <font color='#556b2f'> **Breast Cancer Diagnosis with Support Vector Machines**</font>, 25 points. 

For this problem, we will use the [Wisconsin Breast Cancer](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) data set, which has already been pre-processed and partitioned into training, validation and test sets. Numpy's [loadtxt](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.loadtxt.html) command can be used to load CSV files.

In [ ]:
# Load the Breast Cancer Diagnosis data set; download the files from eLearning
# CSV files can be read easily using np.loadtxt()
#
# Insert your code here.
#

X_trn=np.loadtxt('wdbc_trn.csv', delimiter = ',')
X_test=np.loadtxt('wdbc_tst.csv', delimiter = ',')
X_val=np.loadtxt('wdbc_val.csv', delimiter=',')

XTrain=X_trn[:, 1:]
XTest=X_test[:, 1:]
XVal=X_val[:, 1:]
YTrain=X_trn[:,0]
YTest=X_test[:,0]
YVal=X_val[:,0]

Use scikit-learn's [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) function to learn SVM models with **radial-basis kernels** for **each combination** of $C \in \{10^{-2}, 10^{-1}, 1, 10^1, \, \cdots\, 10^4\}$ and $\gamma \in \{10^{-3}, 10^{-2}\, 10^{-1}, 1, \, 10, \, 10^2\}$. Print the tables corresponding to the training and validation errors.

**Final Model Selection**: Use the validation set to select the best the classifier corresponding to the best parameter values, $C_{best}$ and $\gamma_{best}$. Report the accuracy on the **test set** for this selected best SVM model. _Note: You should report a single number, your final test set accuracy on the model corresponding to $C_{best}$ and $\gamma_{best}$_.

In [ ]:
C_range      = np.arange(-2.0, 5.0, 1.0)
C_values     = np.power(10.0, C_range)
gamma_range  = np.arange(-3.0, 3.0, 1.0)
gamma_values = np.power(10.0, gamma_range)

models             = dict()
trainError         = dict()
validationError    = dict()
target_C, target_G = None,None

minimum=1
for C in C_values:
    for G in gamma_values:
        model = SVC(C=C, kernel = "rbf", gamma = G)
        models[(C,G)]=model.fit(XTrain, YTrain)
        y_pred=model.predict(XVal)
        trainError[(C,G)] = (1- model.score(XTrain, YTrain))
        validationError[(C,G)] = (1- metrics.accuracy_score(YVal, y_pred))
        if validationError[(C,G)] < minimum:
            minimum = validationError[(C, G)]
            target_C = C
            target_G = G
            
predicted = models[target_C, target_G].predict(XTest)
accuracy = metrics.accuracy_score(YTest, predicted)
print("Accuracy: {0:.2f}%, for G = {1:1.3f} and C = {2:1.0f}".format(accuracy * 100,target_G,target_C))

# 3. <font color='#556b2f'> **Breast Cancer Diagnosis with $k$-Nearest Neighbors**</font>, 25 points. 

Use scikit-learn's [k-nearest neighbor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) classifier to learn models for Breast Cancer Diagnosis with $k \in \{1, \, 5, \, 11, \, 15, \, 21\}$, with the kd-tree algorithm.

**Plot**: For each classifier, compute **both** the **training error** and the **validation error**. Plot them together, making sure to label the axes and each curve clearly.

**Final Model Selection**: Use the validation set to select the best the classifier corresponding to the best parameter value, $k_{best}$. Report the accuracy on the **test set** for this selected best kNN model. _Note: You should report a single number, your final test set accuracy on the model corresponding to $k_{best}$_.

In [ ]:
#Model selection
k=[1, 5, 11, 15, 21]

models          = dict()
trainError      = dict()
validationError = dict()
minimum         = 1
target_n        = None

for n in k:
    model = KNeighborsClassifier(n_neighbors= n, algorithm = 'kd_tree')
    models[n] = model.fit(XTrain, YTrain)
    y_pred = model.predict(XVal)
    trainError[n] = (1- models[n].score(XTrain, YTrain))
    validationError[n] = (1- metrics.accuracy_score(YVal, y_pred))
    if validationError[n] < minimum:
        minimum = validationError[n]
        target_n = n
        
predicted = models[target_n].predict(XTest)
accuracy = metrics.accuracy_score(YTest, predicted)
print("Accuracy: {0:.2f}%".format(accuracy * 100))

plt.figure()
plt.plot(list(validationError.keys()), list(validationError.values()), marker='o', linewidth=4, markersize=12)
plt.plot(list(trainError.keys()), list(trainError.values()), marker='s', linewidth=4, markersize=12)
plt.xlabel('No of neighbors', fontsize=18)
plt.ylabel('Validation/Train error', fontsize=18)
plt.xticks(list(validationError.keys()), fontsize=12)
plt.legend(['Validation Error', 'Training Error'], fontsize=18)

We can see that k=5 and k=11 yield the least validation errors.

In [ ]:
neighbor = KNeighborsClassifier(n_neighbors=11,algorithm='kd_tree')
neighbor.fit(XTrain,YTrain) 
y_test_pred=neighbor.predict(XTest)
print('Test Accuracy')
print(accuracy_score(YTest,y_test_pred)*100)

**Discussion**: Which of these two approaches, SVMs or kNN, would you prefer for this classification task? Explain.

We can see that KNN yields a better accuracy (97.39%) as compared to SVM (95.65%). KNN works better with more number of attributes.